https://www.med66.com/sitemap/yixuetiku/1.htm

In [7]:
import re
import os
import requests
import hashlib
import json
import time
import random
import string
import pandas as pd
from tqdm import tqdm
import codecs
from glob import glob
from bs4 import BeautifulSoup
from scrapy.selector import Selector
from urllib.parse import unquote
from concurrent.futures import ThreadPoolExecutor, as_completed

In [8]:
# 定义随机的useragent
from fake_useragent import UserAgent
ua = UserAgent()

# 设置代理ip
ipdata = pd.read_excel('D:/Intern_Project/代理ip的地址2000.xlsx')
ipdata.fillna("",inplace = True)

proxieslist = []

for i in ipdata.index.values:
    line = ipdata.loc[i,['HTTP']].to_dict()
    proxieslist.append(line)
    
# randomproxy = random.choice(proxieslist)

# 设置消息头
headers_pc = {
    'User-Agent': ua.random,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
}

params = {
    'ie': 'UTF-8',
    'wd': 'Python'
}


In [27]:
def changeparams():
    headers_pc = {
    'User-Agent': ua.random,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
    }
    randomproxy = random.choice(proxieslist)
    #  访问百度服务器，获取cookie
#     try:
#         res = requests.get('https://www.med66.com/sitemap/yixuetiku/10.htm', headers=headers_pc, proxies= randomproxy,)
#         # 将cookieJar数据转化为字典，s
#         cookie_dict = requests.utils.dict_from_cookiejar(res.cookies)     
#     except Exception as e:
#         print(e)
#         time.sleep(5)
#         res = requests.get('https://www.med66.com/sitemap/yixuetiku/10.htm', headers=headers_pc, proxies= randomproxy,)
#         # 将cookieJar数据转化为字典，s
#         cookie_dict = requests.utils.dict_from_cookiejar(res.cookies)  
    return headers_pc, randomproxy

# 1、先获取文章链接列表

In [28]:
def get_qa_url(url, headers_pc, randomproxy):
    results = []
    # pbar.set_description(letter)
    try:
        requests.adapters.DEFAULT_RETRIES = 12 # 增加重连次数
        s = requests.session()
        s.keep_alive = False # 关闭多余连接

        r = requests.get(url, headers=headers_pc, proxies=randomproxy, timeout=10)
#             encoding = cchardet.detect(r.content)['encoding']
#             html = r.content.decode(encoding)
        r.encoding = 'utf-8'
        html = r.text
#         if os.path.exists(f"D:/Intern_Project/8_RxList/{tag}_{letter}列表页.html"):   
#             return None
#         with open(f'D:/Intern_Project/13_医学题库/{title}.html', 'w+', encoding='utf-8') as f:
#              f.write(html)

        selector = Selector(text=html)
        qaUrl_list = selector.xpath(
            "//ul[@class='cont']/li/a")
        #print(len(termUrl_list))
        for qaUrl in qaUrl_list:
            qaurl = qaUrl.xpath('./@href').extract()[0]
            title = qaUrl.xpath('./text()').extract()[0]
            # print(term)
            tmp = {
                'title': title,
                'url': qaurl
            }
            results.append(tmp)
            # time.sleep()
        pbar.update(1)
    except Exception as e:
        print(e)
        headers_pc, randomproxy = changeparams()
        pbar.update(1)
    return results

In [29]:
def task_monitor(task_list, mt_pool, results):
    while len(task_list) > 0:
        tmp_task_list = []
        for future in task_list:
            if future.done():
                result = future.result()
                results.extend(result)
            else:
                tmp_task_list.append(future)
        
        task_list = tmp_task_list

In [30]:
pbar = tqdm(total=1635)

# results = []
all_craw_tasks = []
headers_pc, randomproxy = changeparams()
count = 0

with ThreadPoolExecutor(max_workers=10) as mt_pool:
    for page in range(1101, 2736):
        page_url = f'https://www.med66.com/sitemap/yixuetiku/{page}.htm'
        tmp_task = mt_pool.submit(get_qa_url, page_url, headers_pc, randomproxy)
        all_craw_tasks.append(tmp_task)
        count += 1
        if count % 50 == 0:
            time.sleep(1)
            headers_pc, randomproxy = changeparams()

    task_monitor(all_craw_tasks, mt_pool, results)
#     with open(f'D:/Intern_Project/13_医学题库/qa链接列表.json', 'w', encoding='utf-8') as f:
#         f.write(json.dumps(results, indent=2, ensure_ascii=False))



 43%|█████████████████████████████████▊                                             | 700/1635 [03:24<04:33,  3.42it/s]


  0%|                                                                                 | 1/1635 [00:00<09:10,  2.97it/s]

  0%|▏                                                                                | 4/1635 [00:00<02:28, 10.99it/s]

list index out of range




  1%|▍                                                                               | 10/1635 [00:00<00:48, 33.48it/s]

  1%|▍                                                                               | 10/1635 [00:00<00:48, 33.48it/s]

  1%|▍                                                                               | 10/1635 [00:00<00:48, 33.48it/s]

  1%|▋                                                                               | 14/1635 [00:00<01:01, 26.49it/s]

  1%|▉                                                                               | 18/1635 [00:00<00:56, 28.79it/s]

  1%|█▏                                                                              | 23/1635 [00:01<00:56, 28.33it/s]

  1%|█▏                                                                              | 23/1635 [00:01<00:56, 28.33it/s]

  2%|█▎                                                                              | 28/1635 [00:01<01:00, 26.55it/s]

  2%|█▌                       

 10%|███████▋                                                                       | 158/1635 [00:06<02:08, 11.53it/s]

 10%|███████▋                                                                       | 160/1635 [00:06<02:07, 11.53it/s]

 10%|███████▉                                                                       | 163/1635 [00:06<02:34,  9.50it/s]

 10%|███████▉                                                                       | 164/1635 [00:07<02:22, 10.29it/s]

 10%|███████▉                                                                       | 165/1635 [00:07<02:22, 10.29it/s]

 10%|████████                                                                       | 167/1635 [00:07<02:26, 10.03it/s]

 10%|████████                                                                       | 168/1635 [00:07<02:16, 10.76it/s]

 11%|████████▍                                                                      | 174/1635 [00:07<01:41, 14.35it/s]

 11%|████████▌                  

list index out of range


...

 82%|███████████████████████████████████████████████████████████████▋              | 1336/1635 [02:57<10:14,  2.06s/it]

 82%|███████████████████████████████████████████████████████████████▊              | 1337/1635 [02:57<09:58,  2.01s/it]

 82%|███████████████████████████████████████████████████████████████▉              | 1339/1635 [02:57<05:51,  1.19s/it]

 82%|███████████████████████████████████████████████████████████████▉              | 1339/1635 [02:58<05:51,  1.19s/it]

 82%|███████████████████████████████████████████████████████████████▉              | 1339/1635 [02:58<05:51,  1.19s/it]

 82%|███████████████████████████████████████████████████████████████▉              | 1340/1635 [02:58<06:28,  1.32s/it]

 82%|████████████████████████████████████████████████████████████████              | 1342/1635 [02:59<06:20,  1.30s/it]

 82%|████████████████████████████████████████████████████████████████              | 1344/1635 [02:59<05:55,  1.22s/it]

 82%|███████████████████████████

 86%|██████████████████████████████████████████████████████████████████▊           | 1400/1635 [03:19<05:09,  1.32s/it]

 86%|██████████████████████████████████████████████████████████████████▊           | 1401/1635 [03:19<04:15,  1.09s/it]

 86%|██████████████████████████████████████████████████████████████████▊           | 1401/1635 [03:19<04:15,  1.09s/it]

 86%|██████████████████████████████████████████████████████████████████▉           | 1403/1635 [03:19<04:24,  1.14s/it]

 86%|██████████████████████████████████████████████████████████████████▉           | 1404/1635 [03:20<05:06,  1.33s/it]

 86%|███████████████████████████████████████████████████████████████████           | 1405/1635 [03:20<04:45,  1.24s/it]

 86%|███████████████████████████████████████████████████████████████████           | 1407/1635 [03:21<04:25,  1.16s/it]

 86%|███████████████████████████████████████████████████████████████████           | 1407/1635 [03:21<04:25,  1.16s/it]

 86%|███████████████████████████

 90%|█████████████████████████████████████████████████████████████████████▊        | 1464/1635 [03:41<04:32,  1.60s/it]

 90%|█████████████████████████████████████████████████████████████████████▉        | 1466/1635 [03:42<04:07,  1.46s/it]

 90%|█████████████████████████████████████████████████████████████████████▉        | 1467/1635 [03:42<04:11,  1.50s/it]

 90%|██████████████████████████████████████████████████████████████████████        | 1468/1635 [03:43<03:29,  1.25s/it]

 90%|██████████████████████████████████████████████████████████████████████        | 1468/1635 [03:43<03:29,  1.25s/it]

 90%|██████████████████████████████████████████████████████████████████████▏       | 1470/1635 [03:43<04:30,  1.64s/it]

 90%|██████████████████████████████████████████████████████████████████████▏       | 1471/1635 [03:44<04:03,  1.48s/it]

 90%|██████████████████████████████████████████████████████████████████████▏       | 1472/1635 [03:44<04:31,  1.66s/it]

 90%|███████████████████████████

 94%|████████████████████████████████████████████████████████████████████████▉     | 1529/1635 [04:03<03:29,  1.98s/it]

 94%|████████████████████████████████████████████████████████████████████████▉     | 1529/1635 [04:03<03:29,  1.98s/it]

 94%|████████████████████████████████████████████████████████████████████████▉     | 1530/1635 [04:04<03:23,  1.94s/it]

 94%|████████████████████████████████████████████████████████████████████████▉     | 1530/1635 [04:04<03:23,  1.94s/it]

 94%|█████████████████████████████████████████████████████████████████████████     | 1532/1635 [04:04<03:02,  1.77s/it]

 94%|█████████████████████████████████████████████████████████████████████████     | 1532/1635 [04:04<03:02,  1.77s/it]

 94%|█████████████████████████████████████████████████████████████████████████▏    | 1534/1635 [04:04<02:48,  1.67s/it]

 94%|█████████████████████████████████████████████████████████████████████████▏    | 1534/1635 [04:05<02:48,  1.67s/it]

 94%|███████████████████████████

 97%|███████████████████████████████████████████████████████████████████████████▉  | 1592/1635 [04:25<01:16,  1.78s/it]

 97%|███████████████████████████████████████████████████████████████████████████▉  | 1593/1635 [04:26<01:20,  1.92s/it]

 97%|████████████████████████████████████████████████████████████████████████████  | 1594/1635 [04:26<01:19,  1.94s/it]

 98%|████████████████████████████████████████████████████████████████████████████▏ | 1596/1635 [04:26<00:55,  1.42s/it]

 98%|████████████████████████████████████████████████████████████████████████████▏ | 1596/1635 [04:27<00:55,  1.42s/it]

 98%|████████████████████████████████████████████████████████████████████████████▏ | 1596/1635 [04:27<00:55,  1.42s/it]

 98%|████████████████████████████████████████████████████████████████████████████▏ | 1598/1635 [04:28<01:00,  1.63s/it]

 98%|████████████████████████████████████████████████████████████████████████████▎ | 1599/1635 [04:28<01:01,  1.71s/it]

 98%|███████████████████████████

In [32]:
with open(f'D:/Intern_Project/13_医学题库/qa链接列表.json', 'w', encoding='utf-8') as f:
        f.write(json.dumps(results, indent=2, ensure_ascii=False))

In [31]:
print(len(results))

116322


In [53]:
# 用来测试哪些页面能简单获取到
headers_pc, randomproxy = changeparams()
            
requests.adapters.DEFAULT_RETRIES = 12 # 增加重连次数
s = requests.session()
s.keep_alive = False # 关闭多余连接

# url = f'https://www.medicinenet.com/diseases_and_conditions/alpha_u.htm'
url = f'https://www.med66.com/html/tk/13/0929b450106b47f6827fc8784dc24e62.html'
# r = requests.get(url, headers=headers_pc, params=params, proxies=randomproxy, cookies=cookie_dict, timeout=10)
try:  
    # 外网不能加params？？？
    r = requests.get(url, headers=headers_pc, proxies=randomproxy, timeout=10)
    
#     print(r.encoding)
#     print(r.apparent_encoding)
#     r.encoding = r.apparent_encoding
    
    # print(r.content)
    encoding = r.apparent_encoding
    print(encoding)
    html = r.content.decode(encoding)

#     r.encoding = 'utf-8'
#     html = r.text

    with open('D:/Intern_Project/13_医学题库/test乱码.html', 'w+', encoding=encoding) as f:
        f.write(html)
except Exception as e:
    print(e)

# selector = Selector(text=html)
# fullquestion = selector.xpath("//h1[@class='msf']/text()").extract()[0]
# fullcontent = selector.xpath("//div[@id='fontzoom']/text()").extract()[0]
# options = 
# answer = 
# try analysis:
    
# if analysis is not None:
    
    
    
# for termUrl in termUrl_list:
#     url = termUrl.xpath('./@href').extract()[0]
#     term = termUrl.xpath('./text()').extract()[0]
#     # print(term)
#     tmp = {
#         'term': term,
#         'url': url
#     }

# print(tmp)

GB2312


In [49]:
r.content.decode('gbk')

'<!DOCTYPE html>\r\n<html>\r\n<head>\r\n<meta charset="gb2312" />\r\n<title> [临床执业医师实践技能]46岁男性，突发头痛、恶心、呕吐，无明显肢体瘫痪，脑膜刺激征阳性，脑脊液呈均匀一致血性，下列哪一诊断最不可能_医学教育网</title>\r\n<meta name="description" content="选项:A.蛛网膜下腔出血B.脑叶出血C.壳核出血D.尾状核头出血E.小脑出血答案：C解析：答复：壳核出血：系豆纹动脉尤其是其外侧支破裂所致。表现突发的病灶对侧偏瘫、偏深感觉缺失和同向偏盲，双眼球向病灶对侧同向凝视不能，主侧半球可有失语；出血" />\r\n<meta name="keywords" content="46,男性,突发,头痛,恶心,呕吐,临床执业医师实践技能" />\r\n<link href="/css/lanmu/global_new1.css?v=3.0" rel="stylesheet" type="text/css" />\r\n<link href="/css/dialog.css" rel="stylesheet" type="text/css"/>\r\n</head>\r\n<body>\r\n<!--公用页头-->\r\n<script src="/head/head_champion_new1000.js" language="javascript"></script>\r\n<!--公用页头 end-->\r\n<!--头部-->\r\n<div class="container header clearfix">\n\t<div class="logo fl"><a href="https://www.med66.com/" target="_blank" title="医学教育网首页"><img src="/images/linchuangyishi2016/zy_03.png" alt="医学教育网"/></a></div>\n\t\t<h1 class="msf fl"><img src="https://www.med66.com/upload/html/2017/03/zy291514.png" alt="医学教育网"/></h1>\

In [40]:
import cchardet